# Speeding Up the Search for "Compromise Solutions" Before It's Too Late
### A walk through on Surrogate-Assisted MOO (SAMOO)

Now that we know Pareto optimality and that there's a tool to efficiently perform MOO, we have to face another problem: 

For this tutorial, we will still be using the Fonseca-Fleming problem.

In [1]:
import os
import sys
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyemu


Recall that the Fonseca-Fleming problem has two objectives with different decision variable values for their respective minimums

In [2]:
def fonseca_fleming(x):
    x = np.array(x)
    n = len(x)
    # Fonseca-Fleming objective functions
    # f1(x) = 1 - exp(-sum((xi - 1/sqrt(n))^2))
    # f2(x) = 1 - exp(-sum((xi + 1/sqrt(n))^2))
    term1 = np.sum((x - 1/np.sqrt(n))**2)
    term2 = np.sum((x + 1/np.sqrt(n))**2)
    
    obj1 = 1 - np.exp(-term1)
    obj2 = 1 - np.exp(-term2)
    
    return obj1, obj2

n_samples = 1000
x1_range = np.linspace(-4, 4, int(np.sqrt(n_samples)))
x2_range = np.linspace(-4, 4, int(np.sqrt(n_samples)))
x1, x2 = np.meshgrid(x1_range, x2_range)
x1 = x1.flatten()
x2 = x2.flatten()

x1 = x1[:n_samples]
x2 = x2[:n_samples]

n_actual = min(len(x1), len(x2))
objectives = np.array([fonseca_fleming([x1[i], x2[i]]) for i in range(n_actual)])
obj1 = objectives[:, 0]
obj2 = objectives[:, 1]

Let us first generate the template files we need for this tutorial.

In [3]:
base_d = "./base_files"
assert os.path.exists(base_d)

temp_d = "fonseca_fleming_demo"
if os.path.exists(temp_d):
    shutil.rmtree(temp_d)
shutil.copytree(base_d,temp_d)

'fonseca_fleming_demo'

We can easily find Pareto optimal set of decision variables by running PESTPP-MOU. Let's generate an initial swarm size of 50 and perform 20 iterations of MOO. To ensure that ou initial population is evenly distributed and sufficiently samples the decision space, it is advisable to use Latin Hypercube Sampling.

In [4]:
sys.path.append(temp_d)
from LHS_sampler import generate_lhsstarter

os.chdir(temp_d)
generate_lhsstarter(seed=42, n_samples=50, n_dimensions=2)
os.chdir("..")

starter_pop = pd.read_csv(os.path.join(temp_d, "FON_template", "starter.dv_pop.csv"), index_col="real_name")
starter_pop.head()

,x1,x2
real_name,,
gen=0_member=0,-3.328019,-1.181402
gen=0_member=1,-1.866797,-1.402948
gen=0_member=2,1.721195,2.384178
gen=0_member=3,-2.658355,-3.251364
gen=0_member=4,3.365238,1.249119


In [5]:
num_workers = 8
shutil.copy2(os.path.join(temp_d, "FON_template", "starter.dv_pop.csv"),
              os.path.join(temp_d, "FON_template", "initial.dv_pop.csv"))

tmpl_in = os.path.join(temp_d, "FON_template")
sys.path.insert(0,tmpl_in)
from forward_run import ppw_worker as ppw_function
pyemu.os_utils.start_workers(tmpl_in, "pestpp-mou", "fon.pst", num_workers = num_workers,
                             worker_root = temp_d, master_dir = os.path.join(temp_d, "pbm_run"),
                             ppw_function = ppw_function)

Let's plot the resulting Pareto front

In [6]:
from ipywidgets import interact, IntSlider, HBox, VBox, Output
from IPython.display import display
from scipy.interpolate import griddata
import numpy as np
import glob
import matplotlib.pyplot as plt

run_data = pd.read_csv(os.path.join(temp_d, "pbm_run", "fon.pareto.summary.csv"))
max_gen = max(run_data['generation'])

csvfiles = sorted(glob.glob(os.path.join(temp_d, "pbm_run", "*[0-999].dv_pop.csv"), recursive=True), 
                  key=lambda x: int(x.split(".dv")[0].split(".")[1]))
all_dv_list = []
for file in csvfiles:
    generation = int(file.split(".dv")[0].split(".")[1])
    df = pd.read_csv(file).assign(generation=generation)
    df = df[['generation'] + [col for col in df.columns if col != 'generation']] 
    all_dv_list.append(df)
all_dv = pd.concat(all_dv_list, ignore_index=True)

csvfiles = sorted(glob.glob(os.path.join(temp_d, "pbm_run", "*[0-999].obs_pop.csv"), recursive=True), 
                      key=lambda x: int(x.split(".obs")[0].split(".")[1]))

all_obs_list = []
for file in csvfiles:
    generation = int(file.split(".obs")[0].split(".")[1])
    df = pd.read_csv(file).assign(generation=generation)
    df = df[['generation'] + [col for col in df.columns if col != 'generation']] 
    all_obs_list.append(df)
all_obs = pd.concat(all_obs_list, ignore_index=True)

all_data = pd.merge(all_dv, all_obs, on=['generation', 'real_name'])
max_gen = max(all_data['generation'])

x = all_data['x1'].values
y = all_data['x2'].values
z1 = all_data['obj1'].values
z2 = all_data['obj2'].values

xi = np.linspace(min(x), max(x), 100)
yi = np.linspace(min(y), max(y), 100)
xi_grid, yi_grid = np.meshgrid(xi, yi)

zi1 = griddata((x, y), z1, (xi_grid, yi_grid), method='cubic')
zi2 = griddata((x, y), z2, (xi_grid, yi_grid), method='cubic')

out_pareto = Output()
out_obj_space = Output()
out_contour = Output()

def plot_all(generation):

    with out_pareto:
        out_pareto.clear_output(wait=True)
        pareto = run_data.loc[(run_data['generation']==generation) & (run_data['nsga2_front'] == 1)]
        plt.figure(figsize=(6, 4))
        plt.scatter(pareto['obj1'], pareto['obj2'], c='firebrick', s=50, alpha=0.7)
        plt.xlabel('Objective 1')
        plt.ylabel('Objective 2')
        plt.title(f'Pareto Front at Generation {generation}')
        plt.tight_layout()
        plt.show()

    with out_obj_space:
        out_obj_space.clear_output(wait=True)
        all_points = run_data.loc[(run_data['generation']==generation)]
        plt.figure(figsize=(6, 4))
        plt.scatter(all_points['obj1'], all_points['obj2'], edgecolor='green', c = 'none', s=50, alpha=0.7)
        plt.xlabel('Objective 1')
        plt.ylabel('Objective 2')
        plt.title(f'Objective Space at Generation {generation}')
        plt.tight_layout()
        plt.show()

    with out_contour:
        out_contour.clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(8, 6))
        contour1 = ax.contourf(xi_grid, yi_grid, zi1, 15, cmap='plasma', alpha=0.5)
        fig.colorbar(contour1, ax=ax, label='Objective 1')
        contour2 = ax.contour(xi_grid, yi_grid, zi2, 15, cmap='viridis', alpha=0.7)
        fig.colorbar(contour2, ax=ax, label='Objective 2')

        pareto_members = pareto['member'].values
        pareto_dv = all_data[all_data['real_name'].isin(pareto_members)]
        ax.scatter(pareto_dv['x1'], pareto_dv['x2'], edgecolor='firebrick', facecolor='none', s=40, alpha=0.8)
        
        ax.set_xlabel('x1')
        ax.set_ylabel('x2')
        ax.set_title(f'Pareto Optimal Decisions at Generation {generation}')
        ax.grid(True)
        plt.tight_layout()
        plt.show()

generation_slider = IntSlider(min=0, max=max_gen, step=1, value=max_gen, 
                             description='Generation:',
                             continuous_update=False)
plot_all(max_gen)

generation_slider.observe(lambda change: plot_all(change['new']), names='value')
display(VBox([
    generation_slider,
    HBox([out_obj_space, out_pareto]),
    out_contour
]))


As this benchmark is really easy, it doesn't take much time to finish the entire optimization process and it doesn't take a lot of iteration to obtain a set of Pareto optimal solutions. However, in reality, real-world models do not run this fast. Even with parallelisation, it can take a few hundred, even thousand iterations until the swarm converges to the Pareto front. If important timely decisions rely on the outcome of such optimisation process, we would need some help to speed up this process. 

Let's pretend for a while that the Fonseca-Fleming problem is a complex model that is expensive to evaluate. There's nothing we could do about the run time of the complex model. What we can do though is to employ an "emulator", a faster and cheaper model that approximates the relationships between the known (i.e., previously evaluated) input and outputs of the complex model in order to predict its response to some input values that were not previously evaluated. This emulator is also known as the surrogate model. Because the surrogate model is an approximation, its prediction has errors. These errors could mislead the decision-making to some suboptimal solutions that are not actually in the Pareto front. This is some price to pay for speeding up the process, but do'nt worry as we have means to manage these uncertainties and still obtain truly Pareto optimal solutions.

There are many surrogate models available in literature. However, we will use Gaussian Process Regression as it provides a convenient way to take into account the uncertainty in predictions of the surrogate model, which, as previously said, needs to be managed well.

Surrogate-Assisted Multi-Objective Optimisation (SAMOO) follows this general algorithm:
1. LHS sampling of initial training dataset and starting population
2. Complex model evaluation (hereinafter referred to as Outer Iteration)
3. Pareto dominance evaluation
     - if front converged: exit; else: continue to step 4
4. (Re)training the GPR
5. MOO with GPR replacing the complex model (hereinafter referred to as Inner Iterations)
6. Resampling for new training points (also called infills)

Steps 2-6 are performed iteratively until convergence is achieved in Step 3.


Let us first prepare our PEST files (control, template, instruction files) to be SAMOO-ready

In [18]:
sys.path.append(temp_d)
from prep_templates import generate_templates

nmax_inner = 20
os.chdir(temp_d)
generate_templates(nmax_inner)
os.chdir("..")

ImportError: cannot import name 'generate_templates' from 'prep_templates' (c:\Users\rmacasieb\Documents\GitHub\pestpp\benchmarks\MOO_tutorials\fonseca_fleming_demo\prep_templates.py)